In [296]:
# how to interact with multipolygonZM
# what to import
from shapely import wkb
from shapely import wkt
from shapely.geometry import MultiPolygon
from shapely.geometry import Polygon
from shapely.geometry import Point
from shapely.geometry import MultiPoint
from shapely.geometry import MultiLineString
from shapely.geometry import LineString

In [297]:
# import csvs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# read in the data
df = pd.read_csv('../data/estacoes/equacoes_IDF.csv', sep=';')

In [298]:
estacoes = pd.read_csv('../data/rj-cor.clima_pluviometro.estacoes_alertario.csv')

In [299]:
estacoes = estacoes[['id_estacao', 'estacao', 'latitude', 'longitude', 'cota', 'x', 'y', 'endereco', 'situacao']]
estacoes = estacoes[estacoes['situacao'] == 'Operante']
estacoes = estacoes.drop(columns=['situacao'])

In [300]:
# get shp file
import geopandas as gpd
gdf = gpd.read_file('../data/estacoes/equacao_IDF.shp')

In [301]:
gdf_simp = gdf[['geometry', 'globalid', 'a', 'b', 'c', 'd']]

In [302]:
pluviometro = ['Realengo', 'Mendanha', 'Campo Grande', 'Santa Cruz', 'Bangu', 'Irajá', 'Eletrobrás-Taquara', 'Via11', 'Capela Mayrink', 'Jardim Botânico', 'Sabóia Lima', 'Irajá', 'Irajá']
# c do Bangu tá estranho

In [303]:
# new column
gdf_simp = gdf_simp.assign(estacao=pluviometro)

In [304]:
from pyproj import Transformer
transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326")

In [305]:
new_geom = []
for idx, row in gdf_simp.iterrows():
    #print(row['geometry'])
    # divide each by 100
    long_points = row['geometry'].exterior.xy[1]
    long_points = [x for x in long_points]
    lat_points = row['geometry'].exterior.xy[0]
    lat_points = [x for x in lat_points]
    new_polygon = [transformer.transform(x, y) for 
                   x, y in zip(lat_points, long_points)]
    new_polygon = Polygon(new_polygon)
    new_geom.append(new_polygon)
gdf_simp = gdf_simp.assign(new_geom=new_geom)

In [306]:
gdf_simp

,geometry,globalid,a,b,c,d,estacao,new_geom
0,"POLYGON Z ((-4834832.716 -2612656.070 0.000, -...",{50288C1B-F5BE-4E5D-B90E-32799BCAC004},1164.04,0.148,6.96,0.769,Realengo,POLYGON ((-22.839795449698222 -43.432041247642...
1,"POLYGON Z ((-4841630.540 -2617338.310 0.000, -...",{FB017208-19DC-4C7D-B994-1EBF88D28A3D},843.78,0.177,12.00,0.698,Mendanha,POLYGON ((-22.87855333774062 -43.4931071423168...
2,"POLYGON Z ((-4857631.877 -2626765.096 0.000, -...",{E5D67DA0-1BBA-494B-ACAC-50F5AD8EE7AF},891.67,0.187,14.00,0.689,Campo Grande,"POLYGON ((-22.956551286949 -43.63684959465803,..."
3,"POLYGON Z ((-4874779.326 -2620320.591 0.000, -...",{6FCBF184-9382-47D0-ACE1-0C82D19AC270},711.30,0.186,7.00,0.687,Santa Cruz,POLYGON ((-22.903233812075122 -43.790887748963...
4,"POLYGON Z ((-4843776.993 -2619661.352 0.000, -...",{0462CA1A-8C0F-4985-AE9D-A6FA19C079D6},1208.96,0.177,0.14,0.788,Bangu,POLYGON ((-22.89777853129117 -43.5123890612755...
5,"POLYGON Z ((-4813375.407 -2614337.253 0.000, -...",{47181BC1-98A9-4464-92D6-BB9FF7086828},5986.27,0.157,29.70,1.050,Irajá,POLYGON ((-22.8537129468941 -43.23928695953794...
6,"POLYGON Z ((-4839379.305 -2623734.894 0.000, -...",{C7931329-D7B2-4C49-8D72-423E32789AD7},1660.34,0.156,14.79,0.841,Eletrobrás-Taquara,POLYGON ((-22.931484060696075 -43.472883952433...
7,"POLYGON Z ((-4844807.378 -2632844.488 0.000, -...",{6A2DAF9D-A8C1-4FC1-9FD9-74B76D29D55D},1423.30,0.196,14.58,0.796,Via11,POLYGON ((-23.006828811385375 -43.521645167215...
8,"POLYGON Z ((-4822447.835 -2631508.533 0.000, -...",{CDF1FDCE-3A04-440A-98DE-32FF84C3E79E},921.39,0.162,15.46,0.673,Capela Mayrink,POLYGON ((-22.99578186349591 -43.3207859704959...
9,"POLYGON Z ((-4805717.486 -2623430.923 0.000, -...",{8ED38231-ED33-4C59-835E-A66775442BD1},1239.00,0.150,20.00,0.740,Jardim Botânico,POLYGON ((-22.92896921288346 -43.1704946892407...


In [312]:
#iterating over the rows

for index, row in estacoes.iterrows():
    x_  = row['latitude']
    y_ = row['longitude']
    point = Point(x_, y_)
    for index2, row2 in gdf_simp.iterrows():
        #print('Testing if', row['estacao'], 'point', x_, y_,'is in', row2['estacao'])
        if row2['new_geom'].contains(point):
            print(row['id_estacao'], row2['estacao'])

1 Jardim Botânico
2 Jardim Botânico
3 Jardim Botânico
4 Sabóia Lima
5 Sabóia Lima
6 Jardim Botânico
7 Sabóia Lima
9 Irajá
10 Irajá
11 Irajá
12 Bangu
13 Sabóia Lima
14 Eletrobrás-Taquara
15 Jardim Botânico
16 Jardim Botânico
17 Capela Mayrink
18 Via11
19 Via11
20 Campo Grande
21 Via11
22 Santa Cruz
23 Sabóia Lima
24 Irajá
25 Via11
26 Campo Grande
27 Santa Cruz
28 Capela Mayrink
29 Mendanha
30 Via11
31 Jardim Botânico
32 Sabóia Lima
33 Sabóia Lima
